<a href="https://colab.research.google.com/github/shikharkyadesultory/Atomic_Simulation-Eff-mass-Frohlic-omegaLO/blob/main/CMS(ASE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ase

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 17.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from ase.build import bulk
from ase.calculators.emt import EMT
from ase.db import connect
from ase.eos import calculate_eos

db = connect('bulk.db')
for symb in ['Al', 'Ni', 'Cu', 'Pd', 'Ag', 'Pt', 'Au']:
    atoms = bulk(symb, 'fcc')
    atoms.calc = EMT()
    eos = calculate_eos(atoms)
    v, e, B = eos.fit()

    atoms.cell *= (v / atoms.get_volume())**(1 / 3)
    atoms.get_potential_energy()
    db.write(atoms, bm=B)

    print(f"Details for {symb}:")
    print(f"  Volume: {v}")
    print(f"  Energy: {e}")
    print(f"  Bulk Modulus: {B}")

Details for Al:
  Volume: 15.931923303048151
  Energy: -0.004882564299247605
  Bulk Modulus: 0.24913771368349813
Details for Ni:
  Volume: 10.600868900812102
  Energy: -0.013311738418053665
  Bulk Modulus: 1.1046241310550273
Details for Cu:
  Volume: 11.565374510212838
  Energy: -0.00703647873130997
  Bulk Modulus: 0.8392875566772626
Details for Pd:
  Volume: 14.588455633485026
  Energy: -0.0002691205244644834
  Bulk Modulus: 1.1183158315343873
Details for Ag:
  Volume: 16.774808232027375
  Energy: -0.00036686258794560445
  Bulk Modulus: 0.6251315997176192
Details for Pt:
  Volume: 15.079861135270544
  Energy: -0.00014870583541437554
  Bulk Modulus: 1.736401895011947
Details for Au:
  Volume: 16.6835259635432
  Energy: -0.0001350522580452207
  Bulk Modulus: 1.0850176028639944


In [ ]:
from ase.build import add_adsorbate,fcc111
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.db import connect
from ase.optimize import BFGS

db1 = connect('bulk.db')
db2 = connect('ads.db')

def run(symb , a , n ,ads):
  atoms = fcc111(symb,(1,1,n),a=a)
  add_adsorbate(atoms , ads , height = 1.0 , position ='fcc')

  #constraints on all atoms except the adsorbate
  fixed = list(range(len(atoms) - 1))
  atoms.constraints = [FixAtoms(indices=fixed)]

  atoms.calc = EMT()
  opt = BFGS(atoms, logfile = None)
  opt.run(fmax = 0.01)
  potential_energy = atoms.get_potential_energy()
  print(f"Potential energy: {potential_energy}")

  # Print atomic positions
  print("Atomic positions:")
  for atom in atoms:
    print(f"{atom.symbol} {atom.position}")

  return atoms

for row in db1.select():
    a = row.cell[0, 1] * 2
    symb = row.symbols[0]
    for n in [1, 2, 3]:
        for ads in 'CNO':
            atoms = run(symb, a, n, ads)
            db2.write(atoms, layers=n, surf=symb, ads=ads)

Potential energy: 0.6779345525889684
Atomic positions:
Al [0. 0. 0.]
C [1.41220498 0.81533693 1.25584007]
Potential energy: 0.464036583947971
Atomic positions:
Al [0. 0. 0.]
N [1.41220498 0.81533693 1.20937094]
Potential energy: 0.7051746586906389
Atomic positions:
Al [0. 0. 0.]
O [1.41220498 0.81533693 1.54691737]
Potential energy: 0.6823336645147431
Atomic positions:
Al [-5.22620829e-17  1.63067385e+00  0.00000000e+00]
Al [0.         0.         2.30612108]
C [1.41220498 0.81533693 3.56834388]
Potential energy: 0.4770641359148504
Atomic positions:
Al [-5.22620829e-17  1.63067385e+00  0.00000000e+00]
Al [0.         0.         2.30612108]
N [1.41220498 0.81533693 3.52530454]
Potential energy: 0.7048812094815267
Atomic positions:
Al [-5.22620829e-17  1.63067385e+00  0.00000000e+00]
Al [0.         0.         2.30612108]
O [1.41220498 0.81533693 3.85882302]
Potential energy: 0.6774273652982354
Atomic positions:
Al [1.41220498 0.81533693 0.        ]
Al [-5.22620829e-17  1.63067385e+00  2.30

**Formation of ZrS2**

In [ ]:
#Code to form ZrS2 and determine the basic properties of the material
from ase.build import mx2 #to create layered transition metal
from ase.calculators.emt import EMT
from ase.db import connect
formula = 'ZrS2'
structure = '1T' #Octahedral crystal structure
atoms = mx2(formula, kind=structure, a=3.66)#approximate lattice parameters
atoms.calc = EMT()
print(f"Chemical Formula : {atoms.get_chemical_formula()}")
print(f"Number of atoms: {len(atoms)}")
print(f"Lattice vectors:\n{atoms.cell}")
print(f"Atom positions:\n{atoms.get_positions()}")
db = connect('ZrS2.db')

Chemical Formula : S2Zr
Number of atoms: 3
Lattice vectors:
Cell([[3.66, 0.0, 0.0], [-1.83, 3.1696529778510456, 0.0], [0.0, 0.0, 0.0]])
Atom positions:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.83000000e+00  1.05655099e+00  1.59500000e+00]
 [-6.29126381e-18  2.11310199e+00 -1.59500000e+00]]


**Adsorbing Au on the ZrS2 surface**

In [2]:
!apt-get update
!apt-get install -y build-essential libfftw3-dev libxc-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [79.8 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,798 kB]
Get:13 https://r2u.stat.illinois.e

In [3]:
import sys
print(sys.path)

['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython']


In [4]:

!{sys.executable} -m pip uninstall gpaw

In [5]:
!pip3 install gpaw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gpaw: filename=gpaw-25.1.0-cp311-cp311-linux_x86_64.whl size=2822204 sha256=e66c52ec3510bca016addae14cfba416eeb173632ad058141512417ba1f94524
  Stored in directory: /root/.cache/pip/wheels/f5/1e/05/db6758eab2e82f997345fec978af4684e2d16eb1913e4769db
Successfully built gpaw


In [6]:
import sys
!{sys.executable} -m pip install --upgrade --no-cache-dir --prefix /usr/local gpaw

In [ ]:
from ase.build import add_adsorbate
from ase.calculators.emt import EMT
from ase.db import connect
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from ase.build import mx2 # Import mx2 here

formula = 'ZrS2'
structure = '1T' #Octahedral Cyrstal Structure Assume
a_ZrS2 = 3.66 #APPROX
vacuum = 8.0 #adding up to avoid interaction with periodic instants

ZrS2_layer = mx2(formula,kind = structure ,a=a_ZrS2 , size=(2,2,1),vacuum = vacuum)
ZrS2_layer.calc = EMT()
print("ZrS2 layer :")
print(f"Chemical Formula : {ZrS2_layer.get_chemical_formula()}")
print(f"Number of atoms: {len(ZrS2_layer)}")
print(f"Lattice vectors:\n{ZrS2_layer.cell}")
print(f"Atom positions:\n{ZrS2_layer.get_positions()}")

ZrS2 layer :
Chemical Formula : S8Zr4
Number of atoms: 12
Lattice vectors:
Cell([[7.32, 0.0, 0.0], [-3.66, 6.339305955702091, 0.0], [0.0, -0.0, 19.19]])
Atom positions:
[[ 0.00000000e+00  0.00000000e+00  9.59500000e+00]
 [ 1.83000000e+00  1.05655099e+00  1.11900000e+01]
 [-6.29126381e-18  2.11310199e+00  8.00000000e+00]
 [-1.83000000e+00  3.16965298e+00  9.59500000e+00]
 [-2.22044605e-16  4.22620397e+00  1.11900000e+01]
 [-1.83000000e+00  5.28275496e+00  8.00000000e+00]
 [ 3.66000000e+00  0.00000000e+00  9.59500000e+00]
 [ 5.49000000e+00  1.05655099e+00  1.11900000e+01]
 [ 3.66000000e+00  2.11310199e+00  8.00000000e+00]
 [ 1.83000000e+00  3.16965298e+00  9.59500000e+00]
 [ 3.66000000e+00  4.22620397e+00  1.11900000e+01]
 [ 1.83000000e+00  5.28275496e+00  8.00000000e+00]]


In [ ]:
adsorb = 'Au'
adsorp_ht = 2.0
adsorp_st = (0,0) #top position
adsorp_sys = ZrS2_layer.copy() #avoiding adultaration while adsorbing
add_adsorbate(adsorp_sys ,adsorb ,height = adsorp_ht, position = adsorp_st)
print(f"Chemical Formula : {adsorp_sys.get_chemical_formula()}")
print(f"Number of atoms: {len(adsorp_sys)}")
print(f"Atom types :\n{adsorp_sys.get_chemical_symbols()}")
print(f"Atom positions:\n{adsorp_sys[-1].position}")

Chemical Formula : AuS8Zr4
Number of atoms: 13
Atom types :
['Zr', 'S', 'S', 'Zr', 'S', 'S', 'Zr', 'S', 'S', 'Zr', 'S', 'S', 'Au']
Atom positions:
[ 0.    0.   13.19]


In [7]:
!which gpaw

/usr/local/bin/gpaw


In [8]:
import sys
import os
import subprocess

# Find gpaw using which command
try:
    gpaw_exec = subprocess.check_output(['which', 'gpaw']).decode().strip()
except subprocess.CalledProcessError:
    raise FileNotFoundError("Could not locate the gpaw executable. "
                            "Please make sure gpaw is installed and in your PATH.")
gpaw_data_dir = '/usr/local/lib/python3/dist-packages/gpaw/data'

# Create the directories if they don't exist:
os.makedirs(gpaw_data_dir, exist_ok=True)

!{gpaw_exec} install-data {gpaw_data_dir}

Available setups and pseudopotentials
  [*] https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-24.11.0.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-24.1.0.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.20000.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.11271.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.9672.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.8.7929.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.6.6300.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.5.3574.tar.gz

Selected gpaw-setups-24.11.0.tar.gz.  Downloading...
Extracting tarball into /usr/local/lib/python3/dist-packages/gpaw/data
Setups installed into /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0.
Register this setup path in /root/.gpaw/rc.py? [y/n] y
Setup path registered in /root/.gpaw/rc.py.
Current GPAW setup paths in order of search priority:
   1. /usr/local/lib/python

In [9]:
!gpaw info

 -------------------------------------------------------------------------------------------------
| python-3.11.13    /usr/bin/python3                                                              |
| gpaw-25.1.0       /usr/local/lib/python3.11/dist-packages/gpaw/                                 |
| ase-3.25.0        /usr/local/lib/python3.11/dist-packages/ase/                                  |
| numpy-2.0.2       /usr/local/lib/python3.11/dist-packages/numpy/                                |
| scipy-1.15.3      /usr/local/lib/python3.11/dist-packages/scipy/                                |
| libxc-5.1.7       yes                                                                           |
| _gpaw             /usr/local/lib/python3.11/dist-packages/_gpaw.cpython-311-x86_64-linux-gnu.so |
| MPI enabled       yes                                                                           |
| OpenMP enabled    no                                                                            |
|

In [ ]:
!export GPAW_SETUP_PATH=/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0

In [10]:
import os
import ase.io
os.environ['GPAW_SETUP_PATH'] = '~//gpaw-setups-24.11.0'
gpaw_data_path = os.path.join(os.path.dirname(ase.__file__), '..', 'gpaw-data')
# os.environ['GPAW_SETUP_PATH'] = gpaw_data_path

In [11]:
ls /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/

Ag.11.dzp.basis.gz  In.GLLBSC.gz                        Pt.dzp.basis.gz
Ag.11.GLLBSC.gz     In.LDA.gz                           Pt.GLLBSC.gz
Ag.11.LDA.gz        In.PBE.gz                           Pt.LDA.gz
Ag.11.PBE.gz        In.revPBE.gz                        Pt.PBE.gz
Ag.11.revPBE.gz     In.RPBE.gz                          Pt.revPBE.gz
Ag.11.RPBE.gz       I.PBE.gz                            Pt.RPBE.gz
Ag.dzp.basis.gz     Ir.9.dzp.basis.gz                   Rb.dzp.basis.gz
Ag.GLLBSC.gz        Ir.9.GLLBSC.gz                      Rb.GLLBSC.gz
Ag.LDA.gz           Ir.9.LDA.gz                         Rb.LDA.gz
Ag.PBE.gz           Ir.9.PBE.gz                         Rb.PBE.gz
Ag.revPBE.gz        Ir.9.revPBE.gz                      Rb.revPBE.gz
Ag.RPBE.gz          Ir.9.RPBE.gz                        Rb.RPBE.gz
Al.dzp.basis.gz     Ir.dzp.basis.gz                     Re.dzp.basis.gz
Al.GLLBSC.gz        I.revPBE.gz                         Re.GLLBSC.gz
Al.LDA.gz           Ir.GLLBSC.gz         

In [12]:
!gunzip /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.PBE.gz

In [ ]:
ls /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/

Ag.11.dzp.basis.gz  In.GLLBSC.gz                        Pt.dzp.basis.gz
Ag.11.GLLBSC.gz     In.LDA.gz                           Pt.GLLBSC.gz
Ag.11.LDA.gz        In.PBE.gz                           Pt.LDA.gz
Ag.11.PBE.gz        In.revPBE.gz                        Pt.PBE.gz
Ag.11.revPBE.gz     In.RPBE.gz                          Pt.revPBE.gz
Ag.11.RPBE.gz       I.PBE.gz                            Pt.RPBE.gz
Ag.dzp.basis.gz     Ir.9.dzp.basis.gz                   Rb.dzp.basis.gz
Ag.GLLBSC.gz        Ir.9.GLLBSC.gz                      Rb.GLLBSC.gz
Ag.LDA.gz           Ir.9.LDA.gz                         Rb.LDA.gz
Ag.PBE.gz           Ir.9.PBE.gz                         Rb.PBE.gz
Ag.revPBE.gz        Ir.9.revPBE.gz                      Rb.revPBE.gz
Ag.RPBE.gz          Ir.9.RPBE.gz                        Rb.RPBE.gz
Al.dzp.basis.gz     Ir.dzp.basis.gz                     Re.dzp.basis.gz
Al.GLLBSC.gz        I.revPBE.gz                         Re.GLLBSC.gz
Al.LDA.gz           Ir.GLLBSC.gz         

In [ ]:
import os

gpaw_data_path = '/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/'
paw_dataset_file = os.path.join(gpaw_data_path, 'Zr.PBE')

if os.path.exists(paw_dataset_file):
    print(f"The file '{paw_dataset_file}' exists.")
else:
    print(f"The file '{paw_dataset_file}' does not exist.")

The file '/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.PBE' exists.


In [ ]:
!mv /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.PBE /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.PBE.gz

In [ ]:
ls /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/

Ag.11.dzp.basis.gz  In.GLLBSC.gz                        Pt.dzp.basis.gz
Ag.11.GLLBSC.gz     In.LDA.gz                           Pt.GLLBSC.gz
Ag.11.LDA.gz        In.PBE.gz                           Pt.LDA.gz
Ag.11.PBE.gz        In.revPBE.gz                        Pt.PBE.gz
Ag.11.revPBE.gz     In.RPBE.gz                          Pt.revPBE.gz
Ag.11.RPBE.gz       I.PBE.gz                            Pt.RPBE.gz
Ag.dzp.basis.gz     Ir.9.dzp.basis.gz                   Rb.dzp.basis.gz
Ag.GLLBSC.gz        Ir.9.GLLBSC.gz                      Rb.GLLBSC.gz
Ag.LDA.gz           Ir.9.LDA.gz                         Rb.LDA.gz
Ag.PBE.gz           Ir.9.PBE.gz                         Rb.PBE.gz
Ag.revPBE.gz        Ir.9.revPBE.gz                      Rb.revPBE.gz
Ag.RPBE.gz          Ir.9.RPBE.gz                        Rb.RPBE.gz
Al.dzp.basis.gz     Ir.dzp.basis.gz                     Re.dzp.basis.gz
Al.GLLBSC.gz        I.revPBE.gz                         Re.GLLBSC.gz
Al.LDA.gz           Ir.GLLBSC.gz         

In [ ]:
!pip install gpaw ase

!wget https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-24.11.0.tar.gz
!head -n 5 gpaw-setups-24.11.0.tar.gz

--2025-06-18 09:34:19--  https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-24.11.0.tar.gz
Resolving wiki.fysik.dtu.dk (wiki.fysik.dtu.dk)... 130.225.86.27
Connecting to wiki.fysik.dtu.dk (wiki.fysik.dtu.dk)|130.225.86.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71622074 (68M) [application/x-gzip]
Saving to: ‘gpaw-setups-24.11.0.tar.gz’

gpaw-setups-24.11.0 100%[===================>]  68.30M  96.9MB/s    in 0.7s    

2025-06-18 09:34:20 (96.9 MB/s) - ‘gpaw-setups-24.11.0.tar.gz’ saved [71622074/71622074]

����?-���>��>���?H~/��?��ޯ�߾�~�����?��?��OO��oO��������?���������~�_�~���?�����_�������?���[�ߟ���������+�{�+X����x��}�?�~���L������ػ<����t�����|_^��|������g���G����������xW����:���j�����~�7���������� ^5�?<�G��~W������p���=��~��s��s���5���sK����ۿ���}�x� �����Y������~��	� ��o[}���+������ߏY{����d��������~c�ׇ�z���1՗o��o�q�ا�U�O������������9�o����>���z[�#�=}��w�/W�����:7 ��u|g�u��@g�ag����sw��f�l�
��2���w�����:MJ��[�7�@ޏ��

In [ ]:
!tar -xzf gpaw-setups-24.11.0.tar.gz

**CO Adsorption on Cu {For Fun}**

In [ ]:
!pip install gpaw ase

!wget https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-24.11.0.tar.gz

!head -n 5 gpaw-setups-24.11.0.tar.gz

!tar -xzf gpaw-setups-24.11.0.tar.gz

import os
os.environ["GPAW_SETUP_PATH"] = "/content/gpaw-setups-24.11.0"

from ase.build import fcc111, add_adsorbate
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from gpaw import GPAW, PW

slab = fcc111('Cu', size=(2, 2, 3), vacuum=10.0)
add_adsorbate(slab, 'C', height=1.85, position='ontop')
add_adsorbate(slab, 'O', height=3.00, position='ontop')
slab.center(axis=2, vacuum=10.0)

fixed_indices = [atom.index for atom in slab if atom.position[2] < slab.positions[:,2].mean()]
slab.set_constraint(FixAtoms(indices=fixed_indices))

slab.calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts=(2, 2, 1),
    txt='gpaw.log'
)

opt = BFGS(slab, logfile='optimization.log')
opt.run(fmax=0.05)

print("Final energy:", slab.get_potential_energy(), "eV")

--2025-05-07 13:38:00--  https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-24.11.0.tar.gz
Resolving wiki.fysik.dtu.dk (wiki.fysik.dtu.dk)... 130.225.86.27
Connecting to wiki.fysik.dtu.dk (wiki.fysik.dtu.dk)|130.225.86.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71622074 (68M) [application/x-gzip]
Saving to: ‘gpaw-setups-24.11.0.tar.gz.3’

gpaw-setups-24.11.0 100%[===================>]  68.30M  23.4MB/s    in 2.9s    

2025-05-07 13:38:03 (23.4 MB/s) - ‘gpaw-setups-24.11.0.tar.gz.3’ saved [71622074/71622074]

����?-���>��>���?H~/��?��ޯ�߾�~�����?��?��OO��oO��������?���������~�_�~���?�����_�������?���[�ߟ���������+�{�+X����x��}�?�~���L������ػ<����t�����|_^��|������g���G����������xW����:���j�����~�7���������� ^5�?<�G��~W������p���=��~��s��s���5���sK����ۿ���}�x� �����Y������~��	� ��o[}���+������ߏY{����d��������~c�ׇ�z���1՗o��o�q�ا�U�O������������9�o����>���z[�#�=}��w�/W�����:7 ��u|g�u��@g�ag����sw��f�l�
��2���w�����:MJ��[�7�

KeyboardInterrupt: 

In [13]:
!pip install --q ase gpaw

In [14]:
!wget -q https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-24.11.0.tar.gz
!tar -xzf gpaw-setups-24.11.0.tar.gz

In [15]:
import os
os.environ["GPAW_SETUP_PATH"] = "/content/gpaw-setups-24.11.0"

In [16]:
import os

setup_path = os.environ["GPAW_SETUP_PATH"]
zr_pbe_path = os.path.join(setup_path, "Zr.PBE")

if not os.path.isfile(zr_pbe_path):
    raise FileNotFoundError(f"'Zr.PBE' not found in {setup_path}. Please verify the file's presence.")

FileNotFoundError: 'Zr.PBE' not found in /content/gpaw-setups-24.11.0. Please verify the file's presence.

#**ZrS2 Adsorption**
#*Au*


In [18]:
a = 3.682

In [19]:
from ase.build import mx2, add_adsorbate
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from gpaw import GPAW, PW
from ase.io import write
import numpy as np

slab = mx2(formula='ZrS2', kind='1T', a=a, thickness=3.19, vacuum=10)
slab = slab.repeat((2, 2, 1))
slab.center(vacuum=10, axis=2)

s_positions = [atom.position for atom in slab if atom.symbol == 'S']
adsorption_site = s_positions[0][:2]

add_adsorbate(slab, 'Au', height=2.34, position=adsorption_site)

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    txt='au_adsorption.log'
)
slab.calc = calc

write('ZrS2_2x2_Au_adsorbed.cif', slab)
# print("Adsorption energy (eV):", slab.get_potential_energy())

#*Pt*


In [22]:
from ase.build import mx2, add_adsorbate
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from gpaw import GPAW, PW
from ase.io import write
import numpy as np

slab = mx2(formula='ZrS2', kind='1T', a=a, thickness=3.19, vacuum=10)
slab = slab.repeat((2, 2, 1))
slab.center(vacuum=10, axis=2)

s_positions = [atom.position for atom in slab if atom.symbol == 'S']
adsorption_site = s_positions[0][:2]

add_adsorbate(slab, 'Pt', height=2.34, position=adsorption_site)

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    txt='pt_adsorption.log'
)
slab.calc = calc

write('ZrS2_2x2_Pt_adsorbed.cif', slab)
# print("Adsorption energy (eV):", slab.get_potential_energy())

#*Bi*

In [23]:
a = 3.682
c = 17.891
from ase.build import mx2, add_adsorbate
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from gpaw import GPAW, PW
from ase.io import write
import numpy as np

slab = mx2(formula='ZrS2', kind='1T', a=a, thickness=3.19, vacuum=10)
slab = slab.repeat((2, 2, 1))
slab.center(vacuum=10, axis=2)

s_positions = [atom.position for atom in slab if atom.symbol == 'S']
adsorption_site = s_positions[0][:2]

add_adsorbate(slab, 'Bi', height=2.34, position=adsorption_site)

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    txt='bi_adsorption.log'
)
slab.calc = calc

write('ZrS2_2x2_Bi_adsorbed.cif', slab)
# print("Adsorption energy (eV):", slab.get_potential_energy())

#**ZrSe2**
#*Au*

In [24]:
a = 3.795
c = 18.150
slab = mx2(formula='ZrSe2', kind='1T', a=a, thickness=c, vacuum=0)
print("Lattice vectors (Å):")
print(slab.cell.cellpar())
# import os
# os.environ["GPAW_SETUP_PATH"] = "/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0"  # Or your correct path

Lattice vectors (Å):
[  3.795   3.795  18.15   90.     90.    120.   ]


In [25]:
a = 3.795
c = 3.35

slab = mx2(formula='ZrSe2',kind='1T',a=a,thickness=c,vacuum=10)
slab = slab.repeat((2,2,1))
slab.center(vacuum=10,axis=2)

s_positions = [atom.position for atom in slab if atom.symbol == 'Se']
adsorption_site=s_positions[0][:2]
add_adsorbate(slab,'Au',height=2.40,position=adsorption_site)

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode= PW(400),
    xc ='PBE',kpts={'size':(4, 4, 1),'gamma':True},
    txt = 'au_adsorption.log'
)
slab.calc = calc
write('ZrSe2_2x2_Au_adsorption.cif',slab)

#*Pt*

In [26]:
a = 3.795
c = 3.35

slab = mx2(formula='ZrSe2',kind='1T',a=a,thickness=c,vacuum=10)
slab = slab.repeat((2,2,1))
slab.center(vacuum=10,axis=2)

s_positions = [atom.position for atom in slab if atom.symbol == 'Se']
adsorption_site=s_positions[0][:2]
add_adsorbate(slab,'Pt',height=2.40,position=adsorption_site)

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode= PW(400),
    xc ='PBE',
    kpts={'size':(4 ,4 ,1),'gamma':True},
    txt = 'pt_adsorption.log'
)
slab.calc = calc
write('ZrSe2_2x2_Pt_adsorption.cif',slab)

#*Bi*

In [27]:
a = 3.795
c = 3.35

slab = mx2(formula='ZrSe2',kind='1T',a=a,thickness=c,vacuum=10)
slab = slab.repeat((2,2,1))
slab.center(vacuum=10,axis=2)

s_positions = [atom.position for atom in slab if atom.symbol == 'Se']
adsorption_site=s_positions[0][:2]
add_adsorbate(slab,'Bi',height=2.40,position=adsorption_site)

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode= PW(400),
    xc ='PBE',kpts={'size':(4, 4, 1),'gamma':True},
    txt = 'bi_adsorption.log'
)
slab.calc = calc
write('ZrSe2_2x2_Bi_adsorption.cif',slab)



```
# This is formatted as code
```

#**ZrTe2**
#*Au*

In [28]:
a = 4.05
c = 3.65
slab = mx2(formula='ZrTe2', kind='1T', a=a, thickness=c, vacuum=0)
print("Lattice vectors (Å):")
print(slab.cell.cellpar())
# import os
# os.environ["GPAW_SETUP_PATH"] = "/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0"  # Or your correct path

Lattice vectors (Å):
[  4.05   4.05   3.65  90.    90.   120.  ]


In [29]:
slab = mx2(formula='ZrTe2',kind='1T',a=a,thickness=c,vacuum=10)
slab = slab.repeat((2,2,1))
slab.center(vacuum=10,axis=2)

s_positions = [atom.position for atom in slab if atom.symbol == 'Te']
adsorption_site=s_positions[0][:2]
add_adsorbate(slab,'Au',height=2.40,position=adsorption_site)

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode= PW(400),
    xc ='PBE',kpts={'size':(4 ,4 ,1),'gamma':True},
    txt = 'au_adsorption.log'
)
slab.calc = calc
write('ZrTe2_2x2_Au_adsorption.cif',slab)

#*Pt*

In [30]:
slab = mx2(formula='ZrTe2',kind='1T',a=a,thickness=c,vacuum=10)
slab = slab.repeat((2,2,1))
slab.center(vacuum=10,axis=2)

s_positions = [atom.position for atom in slab if atom.symbol == 'Te']
adsorption_site=s_positions[0][:2]
add_adsorbate(slab,'Pt',height=2.40,position=adsorption_site)

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode= PW(400),
    xc ='PBE',kpts={'size':(4, 4, 1),'gamma':True},
    txt = 'au_adsorption.log'
)
slab.calc = calc
write('ZrTe2_2x2_Pt_adsorption.cif',slab)

#*Bi*

In [31]:
slab = mx2(formula='ZrTe2',kind='1T',a=a,thickness=c,vacuum=10)
slab = slab.repeat((2,2,1))
slab.center(vacuum=10,axis=2)

s_positions = [atom.position for atom in slab if atom.symbol == 'Te']
adsorption_site=s_positions[0][:2]
add_adsorbate(slab,'Bi',height=2.40,position=adsorption_site)

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode= PW(400),
    xc ='PBE',kpts={'size':(4 ,4 ,1),'gamma':True},
    txt = 'au_adsorption.log'
)
slab.calc = calc
write('ZrTe2_2x2_Bi_adsorption.cif',slab)

#**ZrS2 (substitution with Au)**

In [32]:
# 1. Installation with proper setup files
# !pip install -q ase gpaw
# !wget -q https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.20000.tar.gz  # Using older but more reliable version
# !tar -xzf gpaw-setups-0.9.20000.tar.gz

import os
os.environ["GPAW_SETUP_PATH"] = "/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/"

# Verify ALL required setup files exist
print("Verifying setup files:")
!find /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/ -name "Zr.*"
!find /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/ -name "Au.*"
!find /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/ -name "S.*"

# 2. Structure creation and substitution
from ase.build import mx2
from gpaw import GPAW, PW
from ase.optimize import BFGS
from ase.visualize import view

# Create ZrS2 monolayer
slab = mx2(formula='ZrS2', kind='1T', a=3.66, thickness=1, vacuum=10.0)

# Visualize before substitution (optional)
print("\nOriginal structure:")
view(slab)

# Perform substitution
zr_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Zr']
substitution_index = zr_indices[0]  # Selecting first Zr atom
slab[substitution_index].symbol = 'Au'  # Actual substitution

print("\nStructure after Au substitution:")
view(slab)  # Verify substitution visually

# 3. DFT calculation setup
slab.calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    setups={'Zr': 'pbe', 'S': 'pbe', 'Au': 'pbe'},  # All lowercase
    txt='substitution.log'
)

# 4. Structure relaxation
opt = BFGS(slab, trajectory='optimization.traj')
try:
    opt.run(fmax=0.05)
    print("\nOptimization successful!")
    print("Final Energy:", slab.get_potential_energy(), "eV")

    # Save final structure
    from ase.io import write
    write('final_structure.cif', slab)
    write('final_structure.traj', slab)  # ASE format for future analysis

except Exception as e:
    print("\nError during optimization:", e)
    print("\nTroubleshooting steps:")
    print("1. Verify setup files exist:")
    !ls /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Au.*
    print("\n2. Try restarting the runtime (Runtime -> Restart runtime) and run again")

Verifying setup files:
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.RPBE.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.LDA.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.dzp.basis.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.PBE
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.revPBE.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.GLLBSC.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Au.dzp.basis.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Au.GLLBSC.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Au.RPBE.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Au.LDA.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Au.revPBE.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Au.PBE.gz
/usr/local/lib/python3/dist-pa

In [ ]:
!ls /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/
# !ls /content/gpaw-setups-0.9.20000/Zr.pbe

Ag.11.dzp.basis.gz  In.GLLBSC.gz			Pt.dzp.basis.gz
Ag.11.GLLBSC.gz     In.LDA.gz				Pt.GLLBSC.gz
Ag.11.LDA.gz	    In.PBE.gz				Pt.LDA.gz
Ag.11.PBE.gz	    In.revPBE.gz			Pt.PBE.gz
Ag.11.revPBE.gz     In.RPBE.gz				Pt.revPBE.gz
Ag.11.RPBE.gz	    I.PBE.gz				Pt.RPBE.gz
Ag.dzp.basis.gz     Ir.9.dzp.basis.gz			Rb.dzp.basis.gz
Ag.GLLBSC.gz	    Ir.9.GLLBSC.gz			Rb.GLLBSC.gz
Ag.LDA.gz	    Ir.9.LDA.gz				Rb.LDA.gz
Ag.PBE.gz	    Ir.9.PBE.gz				Rb.PBE.gz
Ag.revPBE.gz	    Ir.9.revPBE.gz			Rb.revPBE.gz
Ag.RPBE.gz	    Ir.9.RPBE.gz			Rb.RPBE.gz
Al.dzp.basis.gz     Ir.dzp.basis.gz			Re.dzp.basis.gz
Al.GLLBSC.gz	    I.revPBE.gz				Re.GLLBSC.gz
Al.LDA.gz	    Ir.GLLBSC.gz			Re.LDA.gz
Al.PBE.gz	    Ir.LDA.gz				Re.PBE.gz
Al.revPBE.gz	    Ir.PBE.gz				Re.revPBE.gz
Al.RPBE.gz	    I.RPBE.gz				Re.RPBE.gz
Ar.dzp.basis.gz     Ir.revPBE.gz			Rh.9.dzp.basis.gz
Ar.GLLBSC.gz	    Ir.RPBE.gz				Rh.9.GLLBSC.gz
Ar.LDA.gz	    K.dzp.basis.gz			Rh.9.LDA.gz
Ar.PBE.gz	    K.GLLBSC.gz				Rh.9.PBE.gz
Ar.revPBE.gz	    K.LDA.g

#**substituting Zr and S and replacing it with Defect inroduction**

**Au substitution with defect**

In [33]:
from ase.build import mx2
from ase.io import write
from ase.optimize import BFGS
from ase.constraints import FixAtoms
from gpaw import GPAW, PW
import numpy as np


a = 3.682
c = 3.19
vacuum = 15

slab = mx2(formula='ZrS2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((3, 3, 1))
slab.center(vacuum=vacuum, axis=2)

s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'S']
replace_index = s_indices[0]

old_position = slab[replace_index].position.copy()


del slab[replace_index]
slab.append('Au')
slab[-1].position = old_position

z_positions = [atom.z for atom in slab if atom.symbol == 'Zr']
z_mean = np.mean(z_positions)
fixed_indices = [i for i, atom in enumerate(slab) if atom.z < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))


calc = GPAW(
    mode=PW(500),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    occupations={'name': 'fermi', 'width': 0.05},
    spinpol=True,
    txt='ZrS2_Au_defect.log'
)
slab.calc = calc

write('ZrS2_3x3_Au_defect.cif', slab)

**Pt substitution with defect**

In [34]:
vacuum = 15

slab = mx2(formula='ZrS2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((3, 3, 1))
slab.center(vacuum=vacuum, axis=2)

s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'S']
replace_index = s_indices[0]

old_position = slab[replace_index].position.copy()


del slab[replace_index]
slab.append('Pt')
slab[-1].position = old_position

z_positions = [atom.z for atom in slab if atom.symbol == 'Zr']
z_mean = np.mean(z_positions)
fixed_indices = [i for i, atom in enumerate(slab) if atom.z < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))


calc = GPAW(
    mode=PW(500),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    occupations={'name': 'fermi', 'width': 0.05},
    spinpol=True,
    txt='ZrS2_Pt_defect.log'
)
slab.calc = calc

write('ZrS2_3x3_Pt_defect.cif', slab)

**Bi substitution with defect**

In [35]:
vacuum = 15

slab = mx2(formula='ZrS2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((3, 3, 1))
slab.center(vacuum=vacuum, axis=2)

s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'S']
replace_index = s_indices[0]

old_position = slab[replace_index].position.copy()


del slab[replace_index]
slab.append('Bi')
slab[-1].position = old_position

z_positions = [atom.z for atom in slab if atom.symbol == 'Zr']
z_mean = np.mean(z_positions)
fixed_indices = [i for i, atom in enumerate(slab) if atom.z < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))


calc = GPAW(
    mode=PW(500),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    occupations={'name': 'fermi', 'width': 0.05},
    spinpol=True,
    txt='ZrS2_Bi_defect.log'
)
slab.calc = calc

write('ZrS2_3x3_Bi_defect.cif', slab)

#**ZrSe2 substitution of Au,Pt,Bi**

**Au**

In [36]:
from ase.build import mx2
from ase.io import write
from ase.optimize import BFGS
from gpaw import GPAW,PW
import numpy as np

a = 3.795
b = 3.35
vacuum = 15

slab = mx2(formula='ZrSe2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((3, 3, 1))
slab.center(vacuum=vacuum, axis=2)

s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Se']
replace_index = s_indices[0]

old_position = slab[replace_index].position.copy()


del slab[replace_index]
slab.append('Au')
slab[-1].position = old_position

z_positions = [atom.z for atom in slab if atom.symbol == 'Zr']
z_mean = np.mean(z_positions)
fixed_indices = [i for i, atom in enumerate(slab) if atom.z < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))


calc = GPAW(
    mode=PW(500),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    occupations={'name': 'fermi', 'width': 0.05},
    spinpol=True,
    txt='ZrSe2_Au_defect.log'
)
slab.calc = calc

write('ZrSe2_3x3_Au_defect.cif', slab)

In [37]:
slab = mx2(formula='ZrSe2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((3, 3, 1))
slab.center(vacuum=vacuum, axis=2)

s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Se']
replace_index = s_indices[0]

old_position = slab[replace_index].position.copy()


del slab[replace_index]
slab.append('Pt')
slab[-1].position = old_position

z_positions = [atom.z for atom in slab if atom.symbol == 'Zr']
z_mean = np.mean(z_positions)
fixed_indices = [i for i, atom in enumerate(slab) if atom.z < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))


calc = GPAW(
    mode=PW(500),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    occupations={'name': 'fermi', 'width': 0.05},
    spinpol=True,
    txt='ZrSe2_Pt_defect.log'
)
slab.calc = calc

write('ZrSe2_3x3_Pt_defect.cif', slab)

In [38]:
slab = mx2(formula='ZrSe2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((3, 3, 1))
slab.center(vacuum=vacuum, axis=2)

s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Se']
replace_index = s_indices[0]

old_position = slab[replace_index].position.copy()


del slab[replace_index]
slab.append('Bi')
slab[-1].position = old_position

z_positions = [atom.z for atom in slab if atom.symbol == 'Zr']
z_mean = np.mean(z_positions)
fixed_indices = [i for i, atom in enumerate(slab) if atom.z < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))


calc = GPAW(
    mode=PW(500),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    occupations={'name': 'fermi', 'width': 0.05},
    spinpol=True,
    txt='ZrSe2_Bi_defect.log'
)
slab.calc = calc

write('ZrSe2_3x3_Bi_defect.cif', slab)

#**ZrTe2 substitution using Au,Pt,Bi**

**Au**

In [39]:
from ase.build import mx2
from ase.io import write
from ase.optimize import BFGS
from gpaw import GPAW,PW
import numpy as np

a = 4.05
b = 3.65
vacuum = 15

slab = mx2(formula='ZrTe2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((3, 3, 1))
slab.center(vacuum=vacuum, axis=2)

s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Te']
replace_index = s_indices[0]

old_position = slab[replace_index].position.copy()


del slab[replace_index]
slab.append('Au')
slab[-1].position = old_position

z_positions = [atom.z for atom in slab if atom.symbol == 'Zr']
z_mean = np.mean(z_positions)
fixed_indices = [i for i, atom in enumerate(slab) if atom.z < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))


calc = GPAW(
    mode=PW(500),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    occupations={'name': 'fermi', 'width': 0.05},
    spinpol=True,
    txt='ZrTe2_Au_defect.log'
)
slab.calc = calc

write('ZrTe2_3x3_Au_defect.cif', slab)

**Pt**

In [40]:
slab = mx2(formula='ZrTe2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((3, 3, 1))
slab.center(vacuum=vacuum, axis=2)

s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Te']
replace_index = s_indices[0]

old_position = slab[replace_index].position.copy()


del slab[replace_index]
slab.append('Pt')
slab[-1].position = old_position

z_positions = [atom.z for atom in slab if atom.symbol == 'Zr']
z_mean = np.mean(z_positions)
fixed_indices = [i for i, atom in enumerate(slab) if atom.z < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))


calc = GPAW(
    mode=PW(500),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    occupations={'name': 'fermi', 'width': 0.05},
    spinpol=True,
    txt='ZrTe2_Pt_defect.log'
)
slab.calc = calc

write('ZrTe2_3x3_Pt_defect.cif', slab)

**Bi**

In [41]:
slab = mx2(formula='ZrTe2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((3, 3, 1))
slab.center(vacuum=vacuum, axis=2)

s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Te']
replace_index = s_indices[0]

old_position = slab[replace_index].position.copy()


del slab[replace_index]
slab.append('Bi')
slab[-1].position = old_position

z_positions = [atom.z for atom in slab if atom.symbol == 'Zr']
z_mean = np.mean(z_positions)
fixed_indices = [i for i, atom in enumerate(slab) if atom.z < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))


calc = GPAW(
    mode=PW(500),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    occupations={'name': 'fermi', 'width': 0.05},
    spinpol=True,
    txt='ZrTe2_Bi_defect.log'
)
slab.calc = calc

write('ZrTe2_3x3_Bi_defect.cif', slab)

#**ZrS2 replacing S with Au,Pt,Bi**

**Au**

In [42]:
from ase.build import mx2
from gpaw import GPAW,PW
from ase.optimize import BFGS
import numpy as np

a = 3.682
c = 3.19

vacuum = 10

slab = mx2(formula='ZrS2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((2, 2, 1))
slab.center(vacuum=vacuum, axis=2)


s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'S']
au_position = slab[s_indices[0]].position

del slab[s_indices[0]]
slab.append('Au')
slab[-1].position = au_position

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    txt='ZrS2_Au_substitution.log'
)
slab.calc = calc


write('ZrS2_2x2_Au_substituted.cif', slab)

**Pt**

In [43]:
slab = mx2(formula='ZrS2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((2, 2, 1))
slab.center(vacuum=vacuum, axis=2)


s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'S']
au_position = slab[s_indices[0]].position

del slab[s_indices[0]]
slab.append('Pt')
slab[-1].position = au_position

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    txt='ZrS2_Pt_substitution.log'
)
slab.calc = calc


write('ZrS2_2x2_Pt_substituted.cif', slab)

**Bi**

In [44]:
slab = mx2(formula='ZrS2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((2, 2, 1))
slab.center(vacuum=vacuum, axis=2)


s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'S']
au_position = slab[s_indices[0]].position

del slab[s_indices[0]]
slab.append('Bi')
slab[-1].position = au_position

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    txt='ZrS2_Bi_substitution.log'
)
slab.calc = calc


write('ZrS2_2x2_Bi_substituted.cif', slab)

#**ZrSe2 replacing Se with Au,Pt,Bi**

**Au**

In [45]:
from ase.build import mx2
from gpaw import GPAW,PW
from ase.optimize import BFGS
import numpy as np

a = 3.795
b = 3.35

slab = mx2(formula='ZrSe2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((2, 2, 1))
slab.center(vacuum=vacuum, axis=2)


s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Se']
au_position = slab[s_indices[0]].position

del slab[s_indices[0]]
slab.append('Au')
slab[-1].position = au_position

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    txt='ZrSe2_Au_substitution.log'
)
slab.calc = calc


write('ZrSe2_2x2_Au_substituted.cif', slab)

**Pt**

In [46]:
slab = mx2(formula='ZrSe2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((2, 2, 1))
slab.center(vacuum=vacuum, axis=2)


s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Se']
au_position = slab[s_indices[0]].position

del slab[s_indices[0]]
slab.append('Pt')
slab[-1].position = au_position

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    txt='ZrSe2_Pt_substitution.log'
)
slab.calc = calc


write('ZrSe2_2x2_Pt_substituted.cif', slab)

**Bi**

In [47]:
slab = mx2(formula='ZrSe2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((2, 2, 1))
slab.center(vacuum=vacuum, axis=2)


s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Se']
au_position = slab[s_indices[0]].position

del slab[s_indices[0]]
slab.append('Bi')
slab[-1].position = au_position

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    txt='ZrSe2_Bi_substitution.log'
)
slab.calc = calc


write('ZrSe2_2x2_Bi_substituted.cif', slab)

#**ZrTe2 replacing Te with Au,Pt,Bi**

**Au**

In [48]:
from ase.build import mx2
from gpaw import GPAW,PW
from ase.optimize import BFGS
import numpy as np

a = 4.05
b = 3.65

slab = mx2(formula='ZrTe2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((2, 2, 1))
slab.center(vacuum=vacuum, axis=2)


s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Te']
au_position = slab[s_indices[0]].position

del slab[s_indices[0]]
slab.append('Au')
slab[-1].position = au_position

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    txt='ZrTe2_Au_substitution.log'
)
slab.calc = calc


write('ZrTe2_2x2_Au_substituted.cif', slab)

**Pt**

In [49]:
slab = mx2(formula='ZrTe2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((2, 2, 1))
slab.center(vacuum=vacuum, axis=2)


s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Te']
au_position = slab[s_indices[0]].position

del slab[s_indices[0]]
slab.append('Pt')
slab[-1].position = au_position

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    txt='ZrTe2_Pt_substitution.log'
)
slab.calc = calc


write('ZrTe2_2x2_Pt_substituted.cif', slab)

**Bi**

In [50]:
slab = mx2(formula='ZrTe2', kind='1T', a=a, thickness=c, vacuum=vacuum)
slab = slab.repeat((2, 2, 1))
slab.center(vacuum=vacuum, axis=2)


s_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Te']
au_position = slab[s_indices[0]].position

del slab[s_indices[0]]
slab.append('Bi')
slab[-1].position = au_position

z_mean = np.mean([atom.z for atom in slab if atom.symbol == 'Zr'])
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    txt='ZrTe2_Bi_substitution.log'
)
slab.calc = calc


write('ZrTe2_2x2_Bi_substituted.cif', slab)